In [1]:
import os
import gc
import sys
import glob
import time
import math
import numpy as np
import uproot
import pandas
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, AlphaDropout, Activation, BatchNormalization, Flatten, \
                                    Concatenate, PReLU
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, CSVLogger

sys.path.insert(0, "../../python")
from common import *
from DataLoader import DataLoader

In [2]:
model = LoadModel('DeepTau2017v2p3_step1_final.hdf5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
def compile_model(model, learning_rate):
    opt = keras.optimizers.Adam(lr=learning_rate)
    #opt = keras.optimizers.Nadam(lr=learning_rate)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    metrics = [ "accuracy", TauLosses.tau_crossentropy, TauLosses.tau_crossentropy_v2, TauLosses.Le, TauLosses.Lmu,
               TauLosses.Ljet, TauLosses.He, TauLosses.Hmu, TauLosses.Htau, TauLosses.Hjet ]
    model.compile(loss=TauLosses.tau_crossentropy_v2, optimizer=opt, metrics=metrics, weighted_metrics=metrics)

In [4]:
TauLosses.SetSFs(1, 1, 3, 1)
model_name = "DeepTau2017v2p3"
compile_model(model, 1e-3)
#model.summary()

In [5]:
def close_file(f_name):
    file_objs = [ obj for obj in gc.get_objects() if ("TextIOWrapper" in str(type(obj))) and (obj.name == f_name)]
    for obj in file_objs:
        obj.close()

In [6]:
class TimeCheckpoint(Callback):
    def __init__(self, time_interval, file_name_prefix):
        self.time_interval = time_interval
        self.file_name_prefix = file_name_prefix
        self.initial_time = time.time()
        self.last_check_time = self.initial_time
    
    def on_batch_end(self, batch, logs=None):
        if batch % 100 != 0: return
        current_time = time.time()
        delta_t = current_time - self.last_check_time
        if delta_t >= self.time_interval:
            abs_delta_t_h = (current_time - self.initial_time) / 60. / 60.
            self.model.save('{}_b{}_{:.1f}h.h5'.format(self.file_name_prefix, batch, abs_delta_t_h))
            self.last_check_time = current_time

In [7]:
def run_training(train_suffix, model_name, data_loader, epoch, n_epochs):

    train_name = '%s_%s' % (model_name, train_suffix)
    
    cb_acc = []
    for acc_name in ["acc", "weighted_acc"]:
        cb_acc.append(ModelCheckpoint("%s_acc.hdf5" % train_name, monitor="val_%s" % acc_name, save_best_only=True,
                                      save_weights_only=False, mode="max", verbose=1))
    
    losses_names = [ "loss" ]
    for w_suffix in [ "", "weighted" ]:
        for l_name in [ "tau_crossentropy", "tau_crossentropy_v2", "Le", "Lmu", "Ljet", "He", "Hmu", "Htau", "Hjet" ]:
            losses_names.append(w_suffix + l_name)
    cb_losses = []
    
    for loss_name in losses_names:
        cb_losses.append(ModelCheckpoint("%s_%s.hdf5" % (train_name, loss_name), monitor="val_%s" % loss_name,
                                         save_best_only=True, save_weights_only=False, mode="min", verbose=1))

    log_name = "%s.log" % train_name
    if os.path.isfile(log_name):
        close_file(log_name)
        os.remove(log_name)
    csv_log = CSVLogger(log_name, append=True)

    time_checkpoint = TimeCheckpoint(4*60*60, '{}_historic'.format(train_name))
    callbacks = [time_checkpoint, csv_log, *cb_acc, *cb_losses]
    fit_hist = model.fit_generator(data_loader.generator(True), validation_data=data_loader.generator(False),
                                   steps_per_epoch=data_loader.steps_per_epoch, validation_steps=data_loader.validation_steps,
                                   callbacks=callbacks, epochs=n_epochs, initial_epoch=epoch, verbose=1)

    model.save("%s_final.hdf5" % train_name)
    return fit_hist

In [8]:
loader = DataLoader('N:/tau-ml/tuples-v2-t3/training/part_*.h5', netConf_full_cmb, 200, 10000, validation_size=6100000,
                    max_queue_size=100, n_passes=-1)

print(loader.total_size, loader.data_size, loader.validation_size)

72491602 66391602 6100000


In [9]:
fit_hist = run_training('step{}'.format(2), model_name, loader, 0, 2)

Epoch 1/2
 47002/331959 [===>..........................] - ETA: 41:22:21 - loss: 0.2144 - acc: 0.8615 - tau_crossentropy: 0.1498 - tau_crossentropy_v2: 0.2113 - Le: 0.0697 - Lmu: 0.0119 - Ljet: 0.1432 - He: 0.1208 - Hmu: 0.0991 - Htau: 0.0364 - Hjet: 0.1329 - weighted_acc: 0.8616 - weighted_tau_crossentropy: 0.1391 - weighted_tau_crossentropy_v2: 0.2223 - weighted_Le: 0.0866 - weighted_Lmu: 0.0204 - weighted_Ljet: 0.1016 - weighted_He: 0.1527 - weighted_Hmu: 0.1037 - weighted_Htau: 0.0596 - weighted_Hjet: 0.0749

KeyboardInterrupt: 

In [10]:
model.save('v2_diverged.h5')